In [1]:
#!/usr/bin/python
import time
import random 
from pprint import pprint

In [2]:
global BlackBoard 
BlackBoard = {"BATTERY_LEVEL":100, "SPOT":True, "GENERAL":True, "DUST_SPOT":True,
                     "HOME_PATH":"(x1,y1),(x2,y2),(x3,y3)..."}
global runninglist
runninglist = []

In [3]:
class Node:
    pass

In [4]:
class Task(Node):
    def __init__(self,children):
        self.children = children
    

In [5]:
class Task_Status(object):
    FAILURE = 0
    SUCCESS = 1
    RUNNING = 2

In [6]:
class Composite(Node):
    def __init__(self,children):
        self.children = children       

In [7]:
class Priority(Composite):
    def run(self):
        for n in self.children:
            result = n.run()
            if result == Task_Status.RUNNING:
                return Task_Status.RUNNING
            elif result == Task_Status.SUCCESS:
                return Task_Status.SUCCESS
        return Task_Status.FAILURE        
            

In [8]:
class Selector(Composite):
    #runninglist = []
    def run(self): 
        if not runninglist:
            for n in self.children:
                result = n.run()
                if result == Task_Status.RUNNING:
                    return Task_Status.RUNNING
                elif result == Task_Status.SUCCESS:
                    return Task_Status.SUCCESS
            return Task_Status.FAILURE 

        else:
            for x in runninglist:
                index = self.children.index(x)
            for m in self.children[index:]:
                result = m.run()
                if result == Task_Status.RUNNING:
                    runninglist.clear()
                    runninglist.append(m)
                    return Task_Status.RUNNING
                elif result == Task_Status.SUCCESS:
                    runninglist.clear()
                    return Task_Status.SUCCESS               
            runninglist.clear()
            return Task_Status.FAILURE  

In [9]:
class Sequence(Composite):
    #runninglist = []
    def run(self):    
        if not runninglist:
            for n in self.children:              
                result = n.run()
                if result == Task_Status.FAILURE:
                    return Task_Status.FAILURE
                elif result == Task_Status.RUNNING:
                    runninglist.append(n)
                    return Task_Status.RUNNING
            return Task_Status.SUCCESS
        
        else:
            for x in runninglist:
                index = self.children.index(x)
            for m in self.children[index:]:
                result = m.run()
                if result == Task_Status.FAILURE:
                    runninglist.clear()
                    return Task_Status.FAILURE
                elif result == Task_Status.RUNNING:
                    runninglist.clear()
                    runninglist.append(m)
                    return Task_Status.RUNNING
            runninglist.clear()
            return Task_Status.SUCCESS

In [10]:
class Condition(Node):
    #def __init__(self,BB):
     #   self.BB = BlackBoard
    pass

In [11]:
class Baterry_check(Condition):
    def run(self):
        print("BATTERY CHECK!")
        print("------------------------------------")
        time.sleep(1)
        if BlackBoard["BATTERY_LEVEL"] < 30:
            print("BATTERY LEVEL <30%! Need CHARGE!")
            return Task_Status.SUCCESS
        else:
            time.sleep(1)
            print("BATTERY LEVEL >30%!")
            print("------------------------------------")
            time.sleep(1)
            return Task_Status.FAILURE

In [12]:
class Find_hoom(Task):
    def run(self):
        time.sleep(1)
        print("******************************************")
        print("HOME PATH FOUND:"+ BlackBoard["HOME_PATH"]) 
        time.sleep(1)
        return BlackBoard["HOME_PATH"]

In [13]:
class Go_hoom(Task):
    def run(self):
        time.sleep(1)
        print("GOING HOME!")
        return Task_Status.SUCCESS

In [14]:
class Dock(Task):
    def run(self):
        print("ROOMBA IS CHARGING!")
        time.sleep(1)
        BlackBoard["BATTERY_LEVEL"] = 100
        print("CHARGING COMPLETED: Battery_level 100%")
        print("******************************************")
        time.sleep(1)
        return Task_Status.SUCCESS       

In [15]:
class Spot_check(Condition):
    def run(self):
        if BlackBoard["SPOT"] is True:
            time.sleep(1)
            print ("FIND SPOT! START CLEAN!")
            time.sleep(1)
            return Task_Status.SUCCESS
        else:
            return Task_Status.FAILURE

In [16]:
class Decorator(Node):
    def __init__(self,children):
        self.children = children 

In [17]:
class Timer(Decorator):
    def __init__(self, children):
        self.children = children
    def run(self):
        if self.children.run() == Task_Status.SUCCESS:
            return Task_Status.SUCCESS
            

In [18]:
class Clean_spot20(Task):
    def __init__(self, circles):
        self.circles = circles
    def run(self): 
        self.circles = 20
        while(self.circles > 0):
            time.sleep(1)
            print("SPOT CLEANING...")
            self.circles -= 1
            BlackBoard["BATTERY_LEVEL"]-=1
        return Task_Status.SUCCESS

In [19]:
class Clean_spot35(Task):
    def __init__(self, circles):
        self.circles = circles
    def run(self): 
        self.circles = 35
        if BlackBoard["BATTERY_LEVEL"] > 65:
            while(self.circles > 0):
                time.sleep(1)
                print("DUSITY SPOT CLEANING...")
                self.circles -= 1
                BlackBoard["BATTERY_LEVEL"]-=1
            return Task_Status.SUCCESS
        else:
            while(self.circles > 0):
                time.sleep(1)
                print("DUSITY SPOT CLEANING...")
                self.circles -= 1
                BlackBoard["BATTERY_LEVEL"]-=1
            print("NO POWER! PLEASE CHARGE!")
            return Task_Status.FAILURE

In [20]:
class Done_spot(Task):
    def run(self):
        BlackBoard["SPOT"] = False
        time.sleep(1)
        print ("CLEAN SPOT DONE!")
        time.sleep(1)
        print ("BATTERY_LEFT:" + str(BlackBoard["BATTERY_LEVEL"])+"%")
        return Task_Status.SUCCESS

In [21]:
class General(Condition):
    def run(self):
        if BlackBoard["GENERAL"] == True:
            time.sleep(1)
            print ("DO GENERAL CLEANING!")
            time.sleep(1)
            return Task_Status.SUCCESS
        else:
            return Task_Status.FAILURE

In [22]:
class Until_Fail(Decorator):
    def run(self):
        while(self.children.run() == Task_Status.FAILURE):
            return Task_Status.SUCCESS

In [23]:
class Negation(Decorator):
    def run(self):
        result = self.children.run()
        if result == Task_Status.SUCCESS:
            return Task_Status.FAILURE
        else:
            return Task_Status.SUCCESS

In [24]:
class Dusity_spot(Condition):
    def run(self):
        if BlackBoard["DUST_SPOT"] == True:
            time.sleep(1)
            print("FIND DUSITY SPOT! START CLEAN!")
            time.sleep(1)
            return Task_Status.SUCCESS
        else:
            return Task_Status.FAILURE   

In [25]:
class Clean(Task):
    def run(self):
        time.sleep(1)
        print("CLEAN DUSITY SPOT DONE!")
        time.sleep(1)
        return Task_Status.SUCCESS 

In [26]:
class Done_general(Task):
    def run(self):
        BlackBoard["GENERAL"] = False
        time.sleep(1)
        print ("GENERAL CLEAN DONE!")
        time.sleep(1)
        print ("BATTERY LEFT:" + str(BlackBoard["BATTERY_LEVEL"]) + "%")
        

In [27]:
class Do_nothing(Task):
    def run(self):
        time.sleep(1)
        print("DO NOTHING!")
        time.sleep(1)
        return Task_Status.SUCCESS

In [28]:
# build tree
root = Priority(children =[
               Sequence(children = 
                       [Baterry_check(), 
                        Find_hoom(None), 
                        Go_hoom(None), 
                        Dock(None)])
               ,Selector(children = 
                       [Sequence(children = 
                            [Spot_check(), 
                             Timer(children = Clean_spot20(None)),
                             Done_spot(None)])
                        , Sequence(children = 
                            [General(),
                             Sequence(children = 
                                        [Until_Fail(children = 
                                                Sequence(children = 
                                                        [Negation(children = 
                                                                  Baterry_check()),
                                                         Selector(children =[
                                                                Sequence(children = [
                                                                        Dusity_spot(), 
                                                                        Timer(children = 
                                                                              Clean_spot35(None))
                                                                       ]),
                                                                Clean(None)])
                                                        ])
                                                   )
                                      ,Done_general(None)
                                        ]) 
                            ])
                       ]),
                Do_nothing(None)
                ])



In [ ]:
#Randomly generate a variable 'threshold' which value is between 0 and 10. If the threshold is no larger than 1, which
#means the room is very clean, roomba does nothing; If the threshold value is between 2 and 5, which means the room is
#not very dirty, and only needs spot clean or general clean; If the threashold value is between 6 and 10, which means
#the room very dirty, roomba will do dusty spot clean.

while (True):
    threshold = random.randint(0,10)
    if 0 <= threshold <= 1:
        BlackBoard.update({"SPOT": False, "GENERAL": False, "DUST_SPOT":False})
    elif 1 < threshold <= 5:
        BlackBoard.update({"SPOT": True, "GENERAL": True, "DUST_SPOT":False})
    else:
        BlackBoard.update({"SPOT":False,"GENERAL": True, "DUST_SPOT":True})
    
    pprint(BlackBoard)
    print('\n')
    root.run()
    print('\n')
    pprint("#####################################################") 
    

   

{'BATTERY_LEVEL': 100,
 'DUST_SPOT': True,
 'GENERAL': True,
 'HOME_PATH': '(x1,y1),(x2,y2),(x3,y3)...',
 'SPOT': False}


BATTERY CHECK!
------------------------------------
BATTERY LEVEL >30%!
------------------------------------
DO GENERAL CLEANING!
BATTERY CHECK!
------------------------------------
BATTERY LEVEL >30%!
------------------------------------
FIND DUSITY SPOT! START CLEAN!
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY SPOT CLEANING...
DUSITY 